# Trend Lines and Channels Detection

In [52]:
import ccxt
import pandas as pd

In [53]:
tf = '4h'

pair = "atom".upper()

df=pd.read_csv(f"C:\\Users\Grant\Desktop\work_git\work\data\\{tf}\{pair}USDT_{tf}.csv", parse_dates=True)

# df=pd.read_csv(f"C:\\Users\Grant\Desktop\work_git\work\data\\{tf}\{pair}USDT_{tf}.csv", parse_dates=True, index_col='Time')

df

,Time,Open,High,Low,Close,Volume
0,2021-09-15 00:00:00,34.19,35.35,32.61,33.08,1254084.34
1,2021-09-15 04:00:00,33.08,34.39,32.78,34.19,923179.36
2,2021-09-15 08:00:00,34.21,34.41,33.00,33.34,835751.52
3,2021-09-15 12:00:00,33.35,33.76,32.71,33.36,835291.11
4,2021-09-15 16:00:00,33.37,35.05,32.98,33.98,1212385.71
...,...,...,...,...,...,...
994,2022-02-27 16:00:00,28.96,28.99,27.20,27.31,572346.16
995,2022-02-27 20:00:00,27.30,27.69,26.28,26.77,1046045.00
996,2022-02-28 00:00:00,26.77,28.19,26.38,27.46,859931.57
997,2022-02-28 04:00:00,27.47,27.93,26.94,27.74,660607.16


In [54]:

#Check if NA values are in data
df=df[df['Volume']!=0]
df.reset_index(drop=True, inplace=True)
df.isna().sum()
df.tail(10)

,Time,Open,High,Low,Close,Volume
989,2022-02-26 20:00:00,29.87,29.99,29.22,29.88,453192.65
990,2022-02-27 00:00:00,29.88,29.92,28.14,28.47,776247.17
991,2022-02-27 04:00:00,28.49,28.91,28.02,28.64,432487.67
992,2022-02-27 08:00:00,28.62,29.46,28.30,29.37,577586.16
993,2022-02-27 12:00:00,29.37,29.75,28.16,28.95,864211.37
994,2022-02-27 16:00:00,28.96,28.99,27.20,27.31,572346.16
995,2022-02-27 20:00:00,27.30,27.69,26.28,26.77,1046045.00
996,2022-02-28 00:00:00,26.77,28.19,26.38,27.46,859931.57
997,2022-02-28 04:00:00,27.47,27.93,26.94,27.74,660607.16
998,2022-02-28 08:00:00,27.75,28.48,27.75,28.08,331221.11


In [55]:
dfpl = df[:]
import plotly.graph_objects as go
from datetime import datetime

fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['Open'],
                high=dfpl['High'],
                low=dfpl['Low'],
                close=dfpl['Close'])])

fig.layout.xaxis.rangeslider.visible = False
fig.show()

In [56]:
import numpy as np
from matplotlib import pyplot
backcandles= 50
wind = 5

candleid = 997

maxim = np.array([])
minim = np.array([])
xxmin = np.array([])
xxmax = np.array([])
for i in range(candleid-backcandles, candleid+1, wind):
    minim = np.append(minim, df.Low.iloc[i:i+wind].min())
    xxmin = np.append(xxmin, df.Low.iloc[i:i+wind].idxmin())
for i in range(candleid-backcandles, candleid+1, wind):
    maxim = np.append(maxim, df.High.loc[i:i+wind].max())
    xxmax = np.append(xxmax, df.High.iloc[i:i+wind].idxmax())
slmin, intercmin = np.polyfit(xxmin, minim,1)
slmax, intercmax = np.polyfit(xxmax, maxim,1)

dfpl = df[candleid-backcandles:candleid+backcandles]
fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['Open'],
                high=dfpl['High'],
                low=dfpl['Low'],
                close=dfpl['Close'])])
fig.add_trace(go.Scatter(x=xxmin, y=slmin*xxmin + intercmin, mode='lines', name='min slope'))
fig.add_trace(go.Scatter(x=xxmax, y=slmax*xxmax + intercmax, mode='lines', name='max slope'))
fig.layout.xaxis.rangeslider.visible = False
fig.show()

# Fitting intercepts to meet highest or lowest candle point in time slice

In [57]:
dfpl = df[candleid-wind-backcandles:candleid+backcandles]
import plotly.graph_objects as go
from datetime import datetime

fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['Open'],
                high=dfpl['High'],
                low=dfpl['Low'],
                close=dfpl['Close'])])

#-------------------------------------------------------------------------
# Fitting intercepts to meet highest or lowest candle point in time slice
adjintercmin = df.Low.loc[candleid-backcandles:candleid].min() - slmin*df.Low.iloc[candleid-backcandles:candleid].idxmin()
adjintercmax = df.High.loc[candleid-backcandles:candleid].max() - slmax*df.High.iloc[candleid-backcandles:candleid].idxmax()
fig.add_trace(go.Scatter(x=xxmin, y=slmin*xxmin + adjintercmin, mode='lines', name='min slope'))
fig.add_trace(go.Scatter(x=xxmax, y=slmax*xxmax + adjintercmax, mode='lines', name='max slope'))
fig.layout.xaxis.rangeslider.visible=False
fig.show()

# Fitting intercepts to wrap price candles

In [58]:
dfpl = df[candleid-wind-backcandles:candleid+backcandles]
import plotly.graph_objects as go
from datetime import datetime

fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['Open'],
                high=dfpl['High'],
                low=dfpl['Low'],
                close=dfpl['Close'])])

#-------------------------------------------------------------------------
# Fitting intercepts to wrap price candles
adjintercmax = (df.High.iloc[xxmax] - slmax*xxmax).max()
adjintercmin = (df.Low.iloc[xxmin] - slmin*xxmin).min()
fig.add_trace(go.Scatter(x=xxmin, y=slmin*xxmin + adjintercmin, mode='lines', name='min slope'))
fig.add_trace(go.Scatter(x=xxmax, y=slmax*xxmax + adjintercmax, mode='lines', name='max slope'))
fig.layout.xaxis.rangeslider.visible=False

fig.show()

# backcandles time window more dynamic

In [59]:
import numpy as np
from matplotlib import pyplot
backcandles= 40 # 6*8
brange = 10 # backcandles//4 #should be less than backcandles
wind = 6

candleid = 500

optbackcandles= backcandles
sldiff = 997

for r1 in range(backcandles-brange, backcandles+brange):
    maxim = np.array([])
    minim = np.array([])
    xxmin = np.array([])
    xxmax = np.array([])
    for i in range(candleid-r1, candleid+1, wind):
        minim = np.append(minim, df.Low.iloc[i:i+wind].min())
        xxmin = np.append(xxmin, df.Low.iloc[i:i+wind].idxmin())
    for i in range(candleid-r1, candleid+1, wind):
        maxim = np.append(maxim, df.High.loc[i:i+wind].max())
        xxmax = np.append(xxmax, df.High.iloc[i:i+wind].idxmax())
    slmin, intercmin = np.polyfit(xxmin, minim,1)
    slmax, intercmax = np.polyfit(xxmax, maxim,1)
    
    if(abs(slmin-slmax)<sldiff):
        sldiff = abs(slmin-slmax)
        optbackcandles=r1
        slminopt = slmin
        slmaxopt = slmax
        intercminopt = intercmin
        intercmaxopt = intercmax
        maximopt = maxim.copy()
        minimopt = minim.copy()
        xxminopt = xxmin.copy()
        xxmaxopt = xxmax.copy()

        print(optbackcandles)
dfpl = df[candleid-wind-optbackcandles-backcandles:candleid+optbackcandles]  
fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['Open'],
                high=dfpl['High'],
                low=dfpl['Low'],
                close=dfpl['Close'])])

adjintercmax = (df.High.iloc[xxmaxopt] - slmaxopt*xxmaxopt).max()
adjintercmin = (df.Low.iloc[xxminopt] - slminopt*xxminopt).min()
fig.add_trace(go.Scatter(x=xxminopt, y=slminopt*xxminopt + adjintercmin, mode='lines', name='min slope'))
fig.add_trace(go.Scatter(x=xxmaxopt, y=slmaxopt*xxmaxopt + adjintercmax, mode='lines', name='max slope'))
fig.layout.xaxis.rangeslider.visible=False

fig.show()

30
31
33
34
35
39
42
44


In [60]:
import numpy as np
from matplotlib import pyplot
backcandles= 88
brange = 50 #should be less than backcandles
wind = 5

candleid = 997#700

optbackcandles= backcandles
sldiff = 10
sldist = 55
for r1 in range(backcandles-brange, backcandles+brange):
    maxim = np.array([])
    minim = np.array([])
    xxmin = np.array([])
    xxmax = np.array([])
    
    for i in range(candleid-r1, candleid+1, wind):
        minim = np.append(minim, df.Low.iloc[i:i+wind].min())
        xxmin = np.append(xxmin, df.Low.iloc[i:i+wind].idxmin())
    for i in range(candleid-r1, candleid+1, wind):
        maxim = np.append(maxim, df.High.loc[i:i+wind].max())
        xxmax = np.append(xxmax, df.High.iloc[i:i+wind].idxmax())
    slmin, intercmin = np.polyfit(xxmin, minim,1)
    slmax, intercmax = np.polyfit(xxmax, maxim,1)
    
    dist = (slmax*candleid + intercmax)-(slmin*candleid + intercmin)
    if(dist<sldist): #abs(slmin-slmax)<sldiff and
        #sldiff = abs(slmin-slmax)
        sldist = dist
        optbackcandles=r1
        slminopt = slmin
        slmaxopt = slmax
        intercminopt = intercmin
        intercmaxopt = intercmax
        maximopt = maxim.copy()
        minimopt = minim.copy()
        xxminopt = xxmin.copy()
        xxmaxopt = xxmax.copy()

        
print(optbackcandles)
dfpl = df[candleid-wind-optbackcandles-backcandles:candleid+optbackcandles]  
fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['Open'],
                high=dfpl['High'],
                low=dfpl['Low'],
                close=dfpl['Close'])])

adjintercmax = (df.High.iloc[xxmaxopt] - slmaxopt*xxmaxopt).max()
adjintercmin = (df.Low.iloc[xxminopt] - slminopt*xxminopt).min()
fig.add_trace(go.Scatter(x=xxminopt, y=slminopt*xxminopt + adjintercmin, mode='lines', name='min slope'))
fig.add_trace(go.Scatter(x=xxmaxopt, y=slmaxopt*xxmaxopt + adjintercmax, mode='lines', name='max slope'))

fig.layout.xaxis.rangeslider.visible = False

fig.show()

40
